In [3]:
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib
import scipy.io as io
import glob
import sys
import os
import gc

root_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
if root_dir not in sys.path:
    sys.path.append(root_dir)

from src.dn2dem_pos import dn2dem_pos

import astropy.time as atime
from astropy.coordinates import SkyCoord
from astropy import units as u
import sunpy.map

from aiapy.calibrate import degradation
from aiapy.calibrate.util import get_correction_table
from aiapy.calibrate import register, update_pointing


import warnings
warnings.simplefilter('ignore')
matplotlib.rcParams['font.size'] = 16

from sunpy.net import Fido, attrs as a

In [4]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [103]:
# Can either get the example data via fido/vso or adapt the example to your own AIA data set
from sunpy.net import Fido, attrs as a

# Only want the 6 coronal channels - this might also download 304A
wvsrch=a.Wavelength(94*u.angstrom, 335*u.angstrom)

result = Fido.search(a.Time('2024-11-03T12:15:09', '2024-11-03T12:15:19'), a.Instrument("aia"), wvsrch)


In [22]:
# Can either get the example data via fido/vso or adapt the example to your own AIA data set
from sunpy.net import Fido, attrs as a

# Only want the 6 coronal channels - this might also download 304A
wvsrch=a.Wavelength(94*u.angstrom, 335*u.angstrom)

result = Fido.search(a.Time('2021-11-03T12:15:09', '2021-11-03T12:15:19'), a.Instrument("aia"), wvsrch)

In [24]:
files = Fido.fetch(result, path='/home/shwetav/OneDrive/Documents/GitHub/hpc/data_21')

Files Downloaded:   0%|          | 0/7 [03:01<?, ?file/s]2025-10-07 09:21:07 - parfive - INFO: https://sdo7.nascom.nasa.gov/cgi-bin/drms_export.cgi?series=aia__lev1;compress=rice;record=171_1415016947-1415016947 failed to download with exception
Timeout on reading data from socket
2025-10-07 09:21:07 - parfive - INFO: https://sdo7.nascom.nasa.gov/cgi-bin/drms_export.cgi?series=aia__lev1;compress=rice;record=211_1415016948-1415016948 failed to download with exception
Timeout on reading data from socket
2025-10-07 09:21:07 - parfive - INFO: https://sdo7.nascom.nasa.gov/cgi-bin/drms_export.cgi?series=aia__lev1;compress=rice;record=94_1415016950-1415016950 failed to download with exception
Timeout on reading data from socket
2025-10-07 09:21:07 - parfive - INFO: https://sdo7.nascom.nasa.gov/cgi-bin/drms_export.cgi?series=aia__lev1;compress=rice;record=335_1415016951-1415016951 failed to download with exception
Timeout on reading data from socket
2025-10-07 09:21:07 - parfive - INFO: https:

7/0 files failed to download. Please check `.errors` for details


In [5]:
# Load in our AIA files for prep
ff=sorted(glob.glob('/home/shwetav/OneDrive/Documents/GitHub/hpc/data/'+'aia*.fits'))
print(ff)

['/home/shwetav/OneDrive/Documents/GitHub/hpc/data/aia.lev1.131A_2024_11_03T12_15_18.64Z.image_lev1.fits', '/home/shwetav/OneDrive/Documents/GitHub/hpc/data/aia.lev1.171A_2024_11_03T12_15_09.35Z.image_lev1.fits', '/home/shwetav/OneDrive/Documents/GitHub/hpc/data/aia.lev1.193A_2024_11_03T12_15_18.81Z.image_lev1.fits', '/home/shwetav/OneDrive/Documents/GitHub/hpc/data/aia.lev1.211A_2024_11_03T12_15_09.63Z.image_lev1.fits', '/home/shwetav/OneDrive/Documents/GitHub/hpc/data/aia.lev1.304A_2024_11_03T12_15_17.13Z.image_lev1.fits', '/home/shwetav/OneDrive/Documents/GitHub/hpc/data/aia.lev1.335A_2024_11_03T12_15_12.63Z.image_lev1.fits', '/home/shwetav/OneDrive/Documents/GitHub/hpc/data/aia.lev1.94A_2024_11_03T12_15_11.12Z.image_lev1.fits']


In [14]:
from aiapy.calibrate.util import get_pointing_table
from sunpy.time import TimeRange

timerange = TimeRange('2024-11-03T12:15:09', '2024-11-03T12:15:19')
pointing_table = get_pointing_table('jsoc', time_range=timerange)

RuntimeError: No data found for this query: aia.master_pointing3h[2024-11-03T12:15:09.000Z-2024-11-03T12:15:19.000Z], key: **ALL**, seg: None

In [16]:
amaps=sunpy.map.Map(ff)

In [ ]:
times = [atime.Time(m.date) for m in amaps]
time_range = [min(times), max(times)]
print(time_range)

(<Time object: scale='utc' format='isot' value=2024-11-03T12:15:09.350>, <Time object: scale='utc' format='isot' value=2024-11-03T12:15:18.810>)


In [40]:
pointing_table = get_pointing_table("JSOC", time_range=(time_range[0] - 12 * u.h, time_range[0] + 12 * u.h))

In [17]:
# Get the wavelengths of the maps and get index of sort for this list of maps 
wvn0 = [m.meta['wavelnth'] for m in amaps]
print(wvn0)
srt_id = sorted(range(len(wvn0)), key=wvn0.__getitem__)
print(srt_id)

# And now can reorder them
# OK to do it without creating new list as finding order and reordering in same cell
amaps = [amaps[i] for i in srt_id]
print([m.meta['wavelnth'] for m in amaps])

[131, 171, 193, 211, 304, 335, 94]
[6, 0, 1, 2, 3, 4, 5]
[94, 131, 171, 193, 211, 304, 335]


In [21]:
amaps

 SunPy Map
 ---------
 Observatory:		 SDO
 Instrument:		 AIA 4
 Detector:		 AIA
 Measurement:		 94.0 Angstrom
 Wavelength:		 94.0 Angstrom
 Observation Date:	 2024-11-03 12:15:11
 Exposure Time:		 2.902043 s
 Dimension:		 [4096. 4096.] pix
 Coordinate System:	 helioprojective
 Scale:			 [0.600109 0.600109] arcsec / pix
 Reference Pixel:	 [2069.474121 2006.181519] pix
 Reference Coord:	 [0. 0.] arcsec                   
 array([[ 0.  ,  0.  ,  1.  , ..., -0.75,  1.5 , -3.5 ],
        [-1.75,  1.25, -1.  , ...,  0.75,  0.5 ,  1.75],
        [-1.  , -0.75,  2.25, ...,  0.5 , -0.75, -2.25],
        ...,
        [ 2.25, -1.5 ,  0.  , ..., -0.75, -0.75,  1.5 ],
        [ 1.  , -1.  ,  1.25, ...,  1.  ,  0.  , -0.75],
        [-0.5 ,  0.  ,  0.25, ...,  1.75,  1.25,  1.25]],
       shape=(4096, 4096), dtype=float32),
 SunPy Map
 ---------
 Observatory:		 SDO
 Instrument:		 AIA 1
 Detector:		 AIA
 Measurement:		 131.0 Angstrom
 Wavelength:		 131.0 Angstrom
 Observation Date:	 2024-11-03 12:15:

In [ ]:
from datetime import timedelta
from aiapy.calibrate import update_pointing, register
from aiapy.calibrate.util import get_pointing_table  # <-- this fetches the table
# (sometimes located at aiapy.util or aiapy.calibrate.util depending on version)
# If the import above fails, try:
# from aiapy.util import get_pointing_table

# assume you already have `amaps` (list of sunpy.map.Map)

# build a time range covering all your maps
t0 = min(m.date for m in amaps)
t1 = max(m.date for m in amaps) + timedelta(seconds=1)

# fetch the AIA pointing table for that interval (downloads/caches from LMSAL)
aprep = []
for m in amaps:
    m_tmp = update_pointing(m, pointing_table=pointing_table)
    aprep.append(register(m_tmp))

print([m.meta['wavelnth'] for m in aprep])
